In [12]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling1D, Dropout, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.backend import clear_session

import matplotlib.pyplot as plt
from transformer import TransformerBlock, TokenAndPositionEmbedding

In [2]:
!wget https://repo.gorchilov.net/datasets/decoded.tar.gz -O /tmp/decoded.tar.gz
!tar -xf /tmp/decoded.tar.gz -C /tmp

import h5py

train = h5py.File('/tmp/train.h5', mode='r')
test = h5py.File('/tmp/test.h5', mode='r')

--2020-12-16 17:41:18--  https://repo.gorchilov.net/datasets/decoded.tar.gz
Resolving repo.gorchilov.net (repo.gorchilov.net)... 185.97.75.52
Connecting to repo.gorchilov.net (repo.gorchilov.net)|185.97.75.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55598589 (53M) [application/octet-stream]
Saving to: ‘/tmp/decoded.tar.gz’

/tmp/decoded.tar.gz 100%[===================>]  53.02M  10.5MB/s    in 5.1s    

2020-12-16 17:41:23 (10.3 MB/s) - ‘/tmp/decoded.tar.gz’ saved [55598589/55598589]



In [3]:
sample_size = 10000

In [6]:
X_train = train['data'][:sample_size]
y_train = train['labels'][:sample_size]

X_test = test['data'][:sample_size]
y_test = test['labels'][:sample_size]

In [1]:
maxlen = 1000

embed_dim = 4  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 64  # Hidden layer size in feed forward network inside transformer

clear_session()

inputs = Input(shape=(maxlen,))

embedding_layer = TokenAndPositionEmbedding(maxlen, sample_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.1)(x)
outputs = Dense(1, activation='relu')(x)

model = Model(inputs, outputs)

NameError: name 'clear_session' is not defined

model.summary()

In [ ]:
model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit(
    X_train, y_train, batch_size=32, epochs=1000
)

Epoch 1/1000


In [ ]:
start = 0
end = 1000

plt.title('all epochs')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.plot(range(start, end), history.history['loss'][start:end])
plt.show()

In [12]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 5s 15ms/step - loss: 2473.7896


2473.78955078125

In [13]:
model.save('model_saves/transformer-model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model_saves/transformer-model/assets
